# ARPU Forecasting with Prophet

This notebook demonstrates how to use Facebook Prophet for forecasting Average Revenue Per User (ARPU) in a telecom context. We'll build time-series models to predict future revenue and identify trends and seasonal patterns.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# For Prophet
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

# For evaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error
import math

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

## 1. Generate Time Series Data

Since we're working with synthetic data, we'll create a time series dataset that simulates daily ARPU values with trends, seasonality, and noise.

In [ ]:
def generate_arpu_time_series(start_date='2022-01-01', end_date='2024-09-30', base_arpu=65):
    """
    Generate synthetic ARPU time series data with trends and seasonality.
    
    Args:
        start_date: Start date for the time series
        end_date: End date for the time series
        base_arpu: Base ARPU value
    
    Returns:
        DataFrame with date and ARPU columns
    """
    # Create date range
    dates = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Create trend component (gradual increase with some fluctuations)
    days = len(dates)
    trend = np.linspace(0, 0.15, days)  # 15% growth over period
    
    # Add cyclical components
    yearly_cycle = 5 * np.sin(2 * np.pi * np.arange(days) / 365.25)
    quarterly_cycle = 3 * np.sin(2 * np.pi * np.arange(days) / 91.31)
    
    # Weekly seasonality (higher ARPU on weekdays)
    weekly_effect = np.zeros(days)
    for i, date in enumerate(dates):
        day_of_week = date.weekday()
        if day_of_week < 5:  # Monday-Friday
            weekly_effect[i] = 2  # Slightly higher on weekdays
        else:
            weekly_effect[i] = -1  # Lower on weekends
    
    # Monthly patterns (billing cycles)
    monthly_effect = np.zeros(days)
    for i, date in enumerate(dates):
        if date.day <= 7:  # First week of month
            monthly_effect[i] = 3  # Higher after billing
    
    # Add noise
    noise = np.random.normal(0, 2, days)
    
    # Combine components
    arpu_values = base_arpu * (1 + trend) + yearly_cycle + quarterly_cycle + weekly_effect + monthly_effect + noise
    
    # Ensure positive values
    arpu_values = np.maximum(arpu_values, 10)
    
    # Create DataFrame
    df = pd.DataFrame({
        'ds': dates,
        'y': arpu_values
    })
    
    return df

# Generate the time series
arpu_data = generate_arpu_time_series()
print(f"Generated {len(arpu_data)} days of ARPU data")
print(f"Date range: {arpu_data['ds'].min()} to {arpu_data['ds'].max()}")
print(f"ARPU range: ${arpu_data['y'].min():.2f} to ${arpu_data['y'].max():.2f}")
arpu_data.head()

## 2. Visualize the Time Series Data

Let's examine the generated ARPU data to understand its patterns.

In [ ]:
# Plot the time series
plt.figure(figsize=(15, 8))
plt.plot(arpu_data['ds'], arpu_data['y'], linewidth=1, alpha=0.7)
plt.title('Synthetic ARPU Time Series', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('ARPU ($)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Show basic statistics
print("ARPU Statistics:")
print(arpu_data['y'].describe())

## 3. Prepare Data for Prophet

Prophet requires the data to have specific column names: 'ds' for dates and 'y' for values.

In [ ]:
# Prophet requires columns to be named 'ds' and 'y'
prophet_data = arpu_data.copy()
print("Data prepared for Prophet modeling")
prophet_data.head()

## 4. Train Prophet Model

We'll split the data into training and testing sets, then train the Prophet model.

In [ ]:
# Split data into train and test sets (80/20 split)
split_date = prophet_data['ds'].quantile(0.8)
train_data = prophet_data[prophet_data['ds'] <= split_date]
test_data = prophet_data[prophet_data['ds'] > split_date]

print(f"Training data: {len(train_data)} days ({train_data['ds'].min()} to {train_data['ds'].max()})")
print(f"Test data: {len(test_data)} days ({test_data['ds'].min()} to {test_data['ds'].max()})")

# Initialize and configure Prophet model
model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    seasonality_mode='multiplicative',
    changepoint_prior_scale=0.05,  # Flexibility of trend changes
    seasonality_prior_scale=10.0,  # Flexibility of seasonality
    interval_width=0.95  # 95% uncertainty intervals
)

# Add custom seasonalities
model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
model.add_seasonality(name='quarterly', period=91.25, fourier_order=4)

# Fit the model
print("\nTraining Prophet model...")
model.fit(train_data)
print("Model training completed!")

## 5. Make Predictions

Now we'll use the trained model to make predictions on the test set and future dates.

In [ ]:
# Create future dataframe for predictions
future = model.make_future_dataframe(periods=90)  # Predict next 90 days
print(f"Future dataframe created with {len(future)} rows")

# Make predictions
forecast = model.predict(future)
print("Predictions completed!")

# Display forecast results
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

## 6. Evaluate Model Performance

Let's evaluate how well our model performs on the test set.

In [ ]:
# Merge actual values with predictions for test period
test_forecast = forecast[forecast['ds'].isin(test_data['ds'])]
evaluation_df = test_data.merge(test_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on='ds')

# Calculate metrics
mae = mean_absolute_error(evaluation_df['y'], evaluation_df['yhat'])
rmse = math.sqrt(mean_squared_error(evaluation_df['y'], evaluation_df['yhat']))
mape = np.mean(np.abs((evaluation_df['y'] - evaluation_df['yhat']) / evaluation_df['y'])) * 100

print("=== MODEL EVALUATION METRICS ===")
print(f"Mean Absolute Error (MAE): ${mae:.2f}")
print(f"Root Mean Square Error (RMSE): ${rmse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

# Check how many actual values fall within prediction intervals
within_interval = ((evaluation_df['y'] >= evaluation_df['yhat_lower']) & 
                   (evaluation_df['y'] <= evaluation_df['yhat_upper'])).mean() * 100
print(f"Percentage within 95% confidence interval: {within_interval:.1f}%")

## 7. Visualize Results

Let's visualize the forecast along with actual values and components.

In [ ]:
# Plot forecast with actual values
fig, ax = plt.subplots(figsize=(15, 8))

# Plot actual values
ax.plot(train_data['ds'], train_data['y'], 'k-', alpha=0.5, label='Historical (Training)')
ax.plot(test_data['ds'], test_data['y'], 'k-', alpha=0.8, label='Actual (Test)')

# Plot forecast
ax.plot(forecast['ds'], forecast['yhat'], 'b-', label='Forecast')
ax.fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'], 
                color='blue', alpha=0.2, label='95% Confidence Interval')

# Add vertical line to separate train/test
ax.axvline(x=split_date, color='red', linestyle='--', alpha=0.7, label='Train/Test Split')

ax.set_title('ARPU Forecasting with Prophet', fontsize=16)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('ARPU ($)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Analyze Forecast Components

Prophet decomposes the forecast into trend, seasonality, and holiday components.

In [ ]:
# Plot components
fig2 = model.plot_components(forecast)
plt.tight_layout()
plt.show()

## 9. Business Insights

Let's extract some business insights from our forecast.

In [ ]:
# Extract trend component
trend_data = forecast[['ds', 'trend']].copy()
trend_data['year'] = trend_data['ds'].dt.year
trend_data['month'] = trend_data['ds'].dt.month

# Calculate yearly growth rate
yearly_avg = trend_data.groupby('year')['trend'].mean()
growth_rates = yearly_avg.pct_change() * 100

print("=== ARPU TREND ANALYSIS ===")
print("Yearly Average ARPU:")
for year, avg_arpu in yearly_avg.items():
    print(f"  {year}: ${avg_arpu:.2f}")

print("\nYear-over-Year Growth Rates:")
for year, growth in growth_rates.items():
    if not np.isnan(growth):
        print(f"  {year}: {growth:.2f}%")

# Forecast next quarter revenue
next_quarter = forecast[forecast['ds'] > test_data['ds'].max()].head(90)
avg_forecast_arpu = next_quarter['yhat'].mean()
print(f"\nForecasted Average ARPU (Next 90 days): ${avg_forecast_arpu:.2f}")

# Assuming 10,000 customers
estimated_customers = 10000
projected_quarterly_revenue = avg_forecast_arpu * estimated_customers * 90 / 30
print(f"Projected Quarterly Revenue (10k customers): ${projected_quarterly_revenue:,.2f}")

## 10. Model Improvements

Let's explore some ways to improve the model.

In [ ]:
# Add external regressors (hypothetical factors that might affect ARPU)
def add_regressors(df):
    """Add external regressors to the data."""
    df = df.copy()
    
    # Marketing spend (hypothetical)
    df['marketing_spend'] = 5000 + 2000 * np.sin(2 * np.pi * np.arange(len(df)) / 365.25) + \
                           np.random.normal(0, 500, len(df))
    
    # Economic indicator (hypothetical)
    df['economic_index'] = 100 + 5 * np.sin(2 * np.pi * np.arange(len(df)) / 91.25) + \
                          np.random.normal(0, 2, len(df))
    
    return df

# Add regressors to our data
extended_data = add_regressors(prophet_data)

# Split data
train_extended = extended_data[extended_data['ds'] <= split_date]
test_extended = extended_data[extended_data['ds'] > split_date]

# Create enhanced model with regressors
enhanced_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    seasonality_mode='multiplicative'
)

# Add regressors to model
enhanced_model.add_regressor('marketing_spend')
enhanced_model.add_regressor('economic_index')

# Fit enhanced model
print("Training enhanced model with external regressors...")
enhanced_model.fit(train_extended)
print("Enhanced model training completed!")
    
# Create future dataframe with regressors
future_extended = enhanced_model.make_future_dataframe(periods=90)
future_extended = add_regressors(future_extended)
    
# Make predictions with enhanced model
enhanced_forecast = enhanced_model.predict(future_extended)
    
# Compare models
print("\n=== MODEL COMPARISON ===")
print("Base Model MAE:", f"${mae:.2f}")
    
# Calculate MAE for enhanced model
test_enhanced_forecast = enhanced_forecast[enhanced_forecast['ds'].isin(test_data['ds'])]
enhanced_mae = mean_absolute_error(evaluation_df['y'], test_enhanced_forecast['yhat'])
print("Enhanced Model MAE:", f"${enhanced_mae:.2f}")
    
improvement = (mae - enhanced_mae) / mae * 100
print(f"Improvement: {improvement:.2f}%")
    
print("\nARPU forecasting with Prophet completed successfully!")
print("This model can help predict future revenue and identify seasonal patterns.")